In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import gensim
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import csv
from csv import reader
from scipy import spatial
import functools
from collections import Counter
import nltk
from nltk.data import load
#tagdict = load('help/tagsets/upenn_tagset.pickle')

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split

#import researchpy as rp
import scipy.stats as stats

import umap
#import hdbscan
import sklearn.cluster as cluster
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score

from matplotlib.pyplot import figure

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [2]:
%%time

# Load 58 Word2Vec models of MPs in T1 & T2

dictOfModels = {}
folderPath = '/kaggle/input/58alignedmptimemodels/kaggle/working/58-aligned-models-by-mp-and-time'

for file in os.listdir(folderPath):
    filePath = folderPath + '/' + file
    #print(filePath)
    #To accommodate errors while picking up corresponding numpy files of gensim models
    if(len(filePath.split('.'))>1):
        continue
    else:
        model = gensim.models.Word2Vec.load(filePath)
        dictOfModels[file] = model

In [3]:
dictOfModels.keys()

In [4]:
td = dictOfModels['aligned_model_df_t1_1547_Conservative'].wv['brexit']
#print(td.shape)
#print(np.reshape(td,(1,300)).shape)

td = np.reshape(td,(1,300))
#tDf = pd.DataFrame(td)
td

In [5]:
%%time
brexitEmbeddings = {}

for k in dictOfModels.keys():
    
    brexEmb = np.reshape(dictOfModels[k].wv['remain'], (1,300))
        
    time = k.split('df_')[1].split('_')[0]
    mpId = k.split('df_')[1].split('_')[1]
    party = k.split('df_')[1].split('_')[2]
    
    brexEmbDf =pd.DataFrame(brexEmb)
    brexEmbDf['time'] = time
    brexEmbDf['mpId'] = mpId
    brexEmbDf['party'] = party

    
    brexitEmbeddings[k] = brexEmbDf


In [6]:
brexitDf = pd.DataFrame()

for v in brexitEmbeddings.values():
    
    brexitDf = brexitDf.append(v)

brexitDf

In [7]:
times = [int(v.split('t')[1]) for v in brexitDf['time'].values]


In [8]:
%%time
standard_embedding = umap.UMAP(random_state=42).fit_transform(brexitDf.loc[:,0:299])


In [9]:
figure(figsize=(30, 30), dpi=80)

In [10]:
'''plt.scatter(standard_embedding[:, 0], standard_embedding[:, 1], c=times, s=100, cmap='spring',edgecolor='black')
plt.legend(times)'''

fig, ax = plt.subplots(figsize=(10,10))
scatter = ax.scatter(standard_embedding[:, 0], standard_embedding[:, 1], c=times, s=100, cmap='spring',edgecolor='black')

# produce a legend with the unique colors from the scatter
legend1 = ax.legend(*scatter.legend_elements(),loc="upper right", title="Times")
ax.add_artist(legend1)
fig.suptitle('Mp-Time Embeddings of the word $\it{remain}$ in T1 & T2', fontsize=20,)


In [11]:
'''fig, ax = plt.subplots()
for color in times:
    #n = 750
    #x, y = np.random.rand(2, n)
    #scale = 200.0 * np.random.rand(n)
    ax.scatter(standard_embedding[:, 0], standard_embedding[:, 1], c=color, s=100, label=color,
               alpha=30, edgecolors='none')

ax.legend()
ax.grid(True)'''

In [12]:
kmeans_labels = cluster.KMeans(n_clusters=3).fit_predict(brexitDf.loc[:,0:299])
kmeans_labels

In [13]:
len(kmeans_labels)
brexitDf['cluster']=kmeans_labels
brexitDf.reset_index(inplace=True)
brexitDf['cluster'].value_counts()

In [14]:
set(kmeans_labels)

In [15]:
brexitDf

In [16]:
t1 = []
t2 = []
for c in set(kmeans_labels):
    
    if(brexitDf[brexitDf['cluster']==c][brexitDf['time']=='t1'].shape[0]>0):
        t1.append(brexitDf[brexitDf['cluster']==c][brexitDf['time']=='t1'].shape[0])
    else:
        t1.append(0)
        
    if(brexitDf[brexitDf['cluster']==c][brexitDf['time']=='t2'].shape[0]>0):
        t2.append(brexitDf[brexitDf['cluster']==c][brexitDf['time']=='t2'].shape[0])
    else:
        t2.append(0)
        
clusterTimeDf = pd.DataFrame({'t1':t1, 't2':t2})
clusterTimeDf.index.name = 'ClusterID'
clusterTimeDf

In [17]:
#figure(figsize=(10, 10), dpi=80)
#plt.scatter(standard_embedding[:, 0], standard_embedding[:, 1], c=kmeans_labels, s=80, cmap='hsv',edgecolor='b')



fig, ax = plt.subplots(figsize=(10,10))
scatter = ax.scatter(standard_embedding[:, 0], standard_embedding[:, 1], c=kmeans_labels, s=200, cmap='spring',edgecolor='black')

#legend2 = ax.legend(*scatter.legend_elements(),loc="upper right", title="Times")
#ax.add_artist(legend2)
#plt.figure(figsize=(20,20))
# 5 clusters have embeddings only from T2 - top left and middle (18 T2 embeddings in total)
# The largest cluster on the bottom right has 8 T2 embeddings and 32 T1 embeddings 

In [18]:
clusterable_embedding = umap.UMAP(
    n_neighbors=20,
    min_dist=0.0,
    n_components=2,
    random_state=42,
).fit_transform(brexitDf.loc[:,0:299])

In [19]:
fig, ax = plt.subplots(figsize=(8,8))


scatter = ax.scatter(clusterable_embedding[:, 0], clusterable_embedding[:, 1],
            c=times, s=80, cmap='autumn',edgecolor='black')
#plt.figsize=()

# **Clustering by Parties**

In [20]:
brexitDf[brexitDf['cluster']==0]['party'].value_counts()


In [21]:
brexitDf[brexitDf['cluster']==1]['party'].value_counts()


In [22]:
brexitDf[brexitDf['cluster']==2]['party'].value_counts()


In [23]:
t1 = []
t2 = []
for cluster in set(kmeans_labels):
    
    if(brexitDf[brexitDf['cluster']==cluster][brexitDf['time']=='t1'].shape[0]>0):
        t1.append(brexitDf[brexitDf['cluster']==cluster][brexitDf['time']=='t1'].shape[0])
    else:
        t1.append(0)
        
    if(brexitDf[brexitDf['cluster']==cluster][brexitDf['time']=='t2'].shape[0]>0):
        t2.append(brexitDf[brexitDf['cluster']==cluster][brexitDf['time']=='t2'].shape[0])
    else:
        t2.append(0)
        
clusterTimeDf = pd.DataFrame({'t1':t1, 't2':t2})
clusterTimeDf.index.name = 'ClusterID'
clusterTimeDf